In [11]:
!pip install requests-toolbelt

In [1]:
import json
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder
import subprocess
import os
import json
import requests
import zipfile
import csv
import pandas as pd
import time

In [2]:
SERVER = "http://localhost:8000"
#DOWNLOAD_BASE_PATH = "D:/PhD/Model_Building/APKs_and_GitRepos"
#DOWNLOAD_BASE_PATH = "D:/PhD/Model_Building/APKs_and_GitRepos/4_Round"
DOWNLOAD_BASE_PATH = "I:///PhD//Model_Building//APKs_and_GitRepos//4_Round"
MOBSF_BASE_PATH = "D:/PhD/Model_Building/Scanners_and_Decompilers/Scanners/MobSF"
APPLICATION_BASE_PATH = "D:/PhD/Model_Building/Stage_2_Analysing_APKs"
APIKEY = "4a9fc3e94941f832d40975a251056079c9be5406891a934ded21a3c0ecdc5119"

In [3]:
def start_MobSF_server():
    os.chdir(MOBSF_BASE_PATH)
    subprocess.call('start run.bat', shell = True)
    time.sleep(15)

In [4]:
def upload_apk(apk_file):
    print("***************Uploading file***************")
    multipart_data = MultipartEncoder(fields={'file': (apk_file, open(apk_file, 'rb'), 'application/octet-stream')})
    headers = {'Content-Type': multipart_data.content_type, 'Authorization': APIKEY}
    response = requests.post(SERVER + '/api/v1/upload', data=multipart_data, headers=headers)
    print("***************File uploaded***************")
    return response.text

In [5]:
def scan_apk(upload_response):
    print("***************Scanning file***************")
    post_dict = json.loads(upload_response)
    headers = {'Authorization': APIKEY}
    response = requests.post(SERVER + '/api/v1/scan', data=post_dict, headers=headers)
    print("***************File scanned***************")
    return response.text

In [6]:
#This is for delete a scanned file
def delete_scanned_results(scanHash):   
    print("Deleting Scan")
    headers = {'Authorization': APIKEY}
    data = {"hash": scanHash}
    response = requests.post(SERVER + '/api/v1/delete_scan', data=data, headers=headers)
    print(response.text)

In [7]:
def download_reverse_eningeered_files(app_hash, app_name):
    print("***************Downloading re-eningeered files***************")
  
    #downloading file from the MOBSF server
    url = SERVER+'/generate_downloads/?hash='+app_hash+'&file_type=java'
    
    os.chdir(DOWNLOAD_BASE_PATH+"/"+app_name)    
    if (os.path.isdir("MobSF_Reversed")!=True): 
            os.mkdir("MobSF_Reversed")
    os.chdir("MobSF_Reversed")    
    
    java_files_request = requests.get(url, allow_redirects=True)
    open(app_name+".zip", 'wb').write(java_files_request.content)
    
    #extracting the downloaded zip file
    file_path = DOWNLOAD_BASE_PATH+"/"+app_name+"/MobSF_Reversed"
    zip_file_name = app_name+".zip"
    with zipfile.ZipFile(file_path+"/"+zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(file_path)
    
    #deleting the zip file as it is not necessary
    os.remove(zip_file_name)
    
    print("***************Re-eningeered files downloaded***************")

In [8]:
def write_scanned_app_details(app_hash, app_name):
    print("***************Writing data to CSV File***************")
    scanned_App_Details_file = open(APPLICATION_BASE_PATH+'/MobSF_Scanned_App_Details.csv', 'a', newline='')
    scanned_App_Details_file_writer = csv.writer(scanned_App_Details_file)       
    scanned_App_Details_file_writer.writerow([app_name,app_hash])  
    scanned_App_Details_file.close()
    print("***************Completed data writing***************")

In [9]:
def execute_process():
    print("***************Process started***************")
    start_MobSF_server()
    files = os.listdir(DOWNLOAD_BASE_PATH)    
    
    try:
        #Reading scanned apps details
        scanned_apps = pd.read_csv(APPLICATION_BASE_PATH+'/MobSF_Scanned_App_Details.csv')

        for file in files:  
            try:
                print("***************File Name : "+file+".apk***************")
                if(file in scanned_apps.app_name.values):
                    print ("This app is already scanned!")
                else:
                    upload_response = upload_apk(DOWNLOAD_BASE_PATH+"/"+file+"/"+file+".apk")

                    #Retrieveing required information from upload respone
                    json_response = json.loads(upload_response)
                    app_hash = json_response['hash']
                    app_name = json_response['file_name']
                    app_name = app_name[:len(app_name)-4]
                
                    scan_response = scan_apk(upload_response)
                    download_reverse_eningeered_files(app_hash, app_name)
                    write_scanned_app_details(app_hash, app_name)
                    
            except Exception as e:
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                print(e)
                continue
            
    except Exception as ex:
        #File not avaialble means the 1st scan of MobSF
        print("No any scans yet"+ ex)
        
    
    print("***************Process completed***************")

In [11]:
execute_process()

***************Process started***************
***************File Name : olauncher.apk***************
***************Uploading file***************
***************File uploaded***************
***************Scanning file***************
***************File scanned***************
***************Downloading re-eningeered files***************
***************Re-eningeered files downloaded***************
***************Writing data to CSV File***************
***************Completed data writing***************
***************File Name : open-ear.apk***************
This app is already scanned!
***************File Name : open-explorer-beta.apk***************
This app is already scanned!
***************File Name : open-flashlight.apk***************
This app is already scanned!
***************File Name : open-flood.apk***************
This app is already scanned!
***************File Name : open-fndds-viewer.apk***************
This app is already scanned!
***************File Name : open-in-browser.

***************File uploaded***************
***************Scanning file***************
***************File scanned***************
***************Downloading re-eningeered files***************
***************Re-eningeered files downloaded***************
***************Writing data to CSV File***************
***************Completed data writing***************
***************File Name : openmaps.apk***************
***************Uploading file***************
***************File uploaded***************
***************Scanning file***************
***************File scanned***************
***************Downloading re-eningeered files***************
***************Re-eningeered files downloaded***************
***************Writing data to CSV File***************
***************Completed data writing***************
***************File Name : openmensa.apk***************
***************Uploading file***************
***************File uploaded***************
***************Scanning file***

***************File uploaded***************
***************Scanning file***************
***************File scanned***************
***************Downloading re-eningeered files***************
***************Re-eningeered files downloaded***************
***************Writing data to CSV File***************
***************Completed data writing***************
***************File Name : openvegemap.apk***************
***************Uploading file***************
***************File uploaded***************
***************Scanning file***************
***************File scanned***************
***************Downloading re-eningeered files***************
***************Re-eningeered files downloaded***************
***************Writing data to CSV File***************
***************Completed data writing***************
***************File Name : openvpn-for-android.apk***************
***************Uploading file***************
***************File uploaded***************
***************Sca

***************File uploaded***************
***************Scanning file***************
***************File scanned***************
***************Downloading re-eningeered files***************
***************Re-eningeered files downloaded***************
***************Writing data to CSV File***************
***************Completed data writing***************
***************File Name : oscilloscope.apk***************
***************Uploading file***************
***************File uploaded***************
***************Scanning file***************
***************File scanned***************
***************Downloading re-eningeered files***************
***************Re-eningeered files downloaded***************
***************Writing data to CSV File***************
***************Completed data writing***************
***************File Name : osm-dashboard-offline-for-opentracks.apk***************
***************Uploading file***************
***************File uploaded***************


***************File scanned***************
***************Downloading re-eningeered files***************
***************Re-eningeered files downloaded***************
***************Writing data to CSV File***************
***************Completed data writing***************
***************File Name : oversec.apk***************
***************Uploading file***************
***************File uploaded***************
***************Scanning file***************
***************File scanned***************
***************Downloading re-eningeered files***************
***************Re-eningeered files downloaded***************
***************Writing data to CSV File***************
***************Completed data writing***************
***************File Name : oware.apk***************
***************Uploading file***************
***************File uploaded***************
***************Scanning file***************
***************File scanned***************
***************Downloading re-eningee

***************File uploaded***************
***************Scanning file***************
***************File scanned***************
***************Downloading re-eningeered files***************
***************Re-eningeered files downloaded***************
***************Writing data to CSV File***************
***************Completed data writing***************
***************Process completed***************
